In [4]:
import datetime as dt

import pandas as pd
import numpy as np
from highcharts import Highchart

data_dir = '../data/'

In [102]:
# Read file, parsing dates
parser = lambda date: pd.datetime.strptime(date, '%d-%m-%Y')
f = pd.read_csv(data_dir + 'transactions.csv', parse_dates=['Date'], date_parser=parser)

# Rename columns for convenience
columns1 = {
  c: c.lower().strip().replace(' ', '_')
  for c in f.columns
  }
columns2 = {
  'memo/description': 'memo',
  'amount_(credit)': 'credit',
  'amount_(debit)': 'debit',
  }
f = f.rename(columns=columns1).rename(columns=columns2)

# Select non-grant deposits
cond1 = f['credit'] > 0  # Some credit
cond2 = ~f['memo'].str.contains('grant', case=False) # Exclude grants
mask = cond1 & cond2
g = f[mask].copy()

# Create deposits time series
deposits = g[['date', 'credit']].copy()
deposits['date'] = pd.to_datetime(deposits['date'])
deposits = deposits.set_index('date')

# Resample to month
monthly_deposits = deposits.resample('M').sum().fillna(0)

# Compute cumulative movining average
cma = monthly_deposits.expanding(min_periods=1).mean() 

monthly_deposits

,credit
date,
2014-05-31,75.0
2014-06-30,40.0
2014-07-31,155.5
2014-08-31,124.5
2014-09-30,246.9
2014-10-31,0.0
2014-11-30,160.0
2014-12-31,100.0
2015-01-31,0.0


In [103]:
# Plot with Highcharts

chart = Highchart()
options = {
    'chart': {
        'width': 700,
    },
    'title': {
        'text': 'Tumeke Cycle Space Deposits'
    },
    'xAxis': {
        'type': 'datetime',
        'labels': {
            'format': '{value:%Y-%m}',
            'rotation': -45,
            'align': 'right',
        },
        'tickInterval': 28*24*3600*1000,
    },
    'yAxis': {
        'title': {
            'text': ''
        },
        'labels': {
            'formatter': "function () {\
                return '$' + this.value;\
            }"
        },
        'lineWidth': 2
    },
    'tooltip': {
        'headerFormat': '<b>{series.name}</b><br/>',
        'pointFormat': '{point.x: %Y-%m}, ${point.y:.0f}',
    }
}
chart.set_dict_options(options)
data1 =  monthly_deposits.reset_index().values.tolist()
chart.add_data_set(data1, 'column', 'Deposits') 
data2 =  cma.reset_index().values.tolist()
chart.add_data_set(data2, 'spline', 'Cumulative Moving Average') 
chart